# Isochrone interpolation

In [0]:
PROJECT_PATH = '../../python'
ISOCHRONES_PATH = '/datascope/subaru/data/isochrones/mist/import/MIST_v1.2_vvcrit0.0_HSC.h5'

MAG1, MAG2, MAG3 = 'hsc_g', 'hsc_i', 'hsc_i'

In [0]:
%matplotlib inline

In [0]:
import os, sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [0]:
plt.rc('font', size=7)

In [0]:
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

In [0]:
tf.config.list_physical_devices('CPU')

In [0]:
tf.config.list_physical_devices('GPU')

In [0]:
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

## Load grid from HDF5

In [0]:
from pfs.ga.isochrones import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.expandvars(ISOCHRONES_PATH))

In [0]:
iso.axes.keys()

In [0]:
for k in iso.values.keys():
    print(k, 
          tf.math.count_nonzero(tf.math.is_inf(iso.values[k])).numpy(),
          tf.math.count_nonzero(tf.math.is_nan(iso.values[k])).numpy())

## Interpolate isochrones

In [0]:
M_ini = tf.convert_to_tensor(np.linspace(0.8, 2.0, 100))
log_t = tf.convert_to_tensor(np.full_like(M_ini, 9.0))
Fe_H = tf.convert_to_tensor(np.full_like(M_ini, 0.0))

In [0]:
mi_EEP, mi_M_ini, [hsc_g, hsc_i], mask = iso.interp3d(Fe_H, log_t, M_ini, [iso.values['hsc_g'], iso.values['hsc_i']])

In [0]:
f, ax = plt.subplots(1, 1)

ax.plot(hsc_g - hsc_i, hsc_i, '.')
ax.invert_yaxis()